# UR5 Install

We have to install the driver and the arm description packages:
- driver: https://github.com/UniversalRobots/Universal_Robots_ROS_Driver
- Arm description package: https://github.com/fmauch/universal_robot/tree/calibration_devel

This driver requires a system setup with ROS. It is recommended to use Ubuntu 18.04 with ROS melodic, however using Ubuntu 16.04 with ROS kinetic should also work.

The driver consists of a C++ library that abstracts the robot's interfaces and a ROS driver on top of that.

The instructions to build our workspace are (substitute "catkin_ws" by your workspace name):

In [ ]:
# create a catkin workspace
$ mkdir -p catkin_ws/src && cd catkin_ws

# clone the driver
$ git clone https://github.com/UniversalRobots/Universal_Robots_ROS_Driver.git src/Universal_Robots_ROS_Driver

# clone fork of the description. This is currently necessary, until the changes are merged upstream.
$ git clone -b calibration_devel https://github.com/fmauch/universal_robot.git src/fmauch_universal_robot

# install dependencies
$ sudo apt update -qq
$ rosdep update
$ rosdep install --from-paths src --ignore-src -y

# build the workspace. We need an isolated build because of the non-catkin library package.
$ catkin_make

# activate the workspace (ie: source it)
$ source devel/setup.bash

Verify the lasts lines in .bashrc file

In [ ]:
source /opt/ros/melodic/setup.bash
source ~/ur5arm_ws/devel/setup.bash
export ROS_IP=192.168.18.92
export ROS_MASTER_URI=http://localhost:11311


### Connection PC to UR

We connect UR5 and PC with Ethernet cable

Assign IP static address:

- 192.168.1.1 to PC

- try ifconfig first and take the interface name:enp2s0 (can change with PCs)

In [ ]:
sudo ip addr add 192.168.1.1/24 dev enp2s0

Define IP address to UR5:

- 192.168.1.2

## Prepare the robot
For using the ur_robot_driver with a real robot you need to install the "externalcontrol-1.0.4.urcap" which can be found inside the resources folder of this driver.

Follow the instructions to install this URCAP: https://github.com/UniversalRobots/Universal_Robots_ROS_Driver/blob/master/ur_robot_driver/doc/install_urcap_e_series.md

## Prepare the ROS PC
Each UR robot is calibrated inside the factory giving exact forward and inverse kinematics. To also make use of this in ROS, you first have to extract the calibration information from the robot.

Though this step is not necessary to control the robot using this driver, it is highly recommended to do so, as otherwise endeffector positions might be off in the magnitude of centimeters.

Callibration the real robot, run:

In [ ]:
roslaunch ur_calibration calibration_correction.launch \
  robot_ip:=<robot_ip> target_filename:="${HOME}/my_robot_calibration.yaml"

For the parameter robot_ip insert the IP address on which the ROS pc can reach the robot. As target_filename provide an absolute path where the result will be saved to.

### Quick start
If you calibrated your robot before, pass that calibration to the launch file:

In [ ]:
roslaunch ur_robot_driver ur5e_bringup.launch robot_ip:=192.168.1.2 \
  kinematics_config:="${HOME}/my_robot_calibration.yaml"

Once the robot driver is started, load the previously generated program on the robot panel that will start the External Control program node and execute it. From that moment on the robot is fully functional. You can make use of the Pause function or even Stop (⏹️) the program. Simply press the Play button (▶️) again and the ROS driver will reconnect.

You are ready to test:
- Forward Kinematics 
- Inverse Kinematics

### Forward Kinematics
To control each joint angle, you need to install the pkg: rqt_joint_trajectory_controller


In [ ]:
sudo apt-get install ros-melodic-rqt-joint-trajectory-controller

In [ ]:
rosrun rqt_joint_trajectory_controller rqt_joint_trajectory_controller

### Inverse Kinematics
To place robot arm in a speciffic POSE you can:
- use moveIt!
- create a specific code in python

IK with moveIt!

https://github.com/ros-industrial/universal_robot/tree/melodic-devel

First launch the driver:


In [ ]:
roslaunch ur_robot_driver ur5e_bringup.launch robot_ip:=192.168.1.2 \
  kinematics_config:="${HOME}/my_robot_calibration.yaml"

Now you can find the "controllers.yaml" file in "/home/mpuig/ur5arm_ws/src/fmauch_universal_robot/ur5_e_moveit_config/config/controllers.yaml" and perform a modification:
- The action_ns has to be changed from: follow_joint_trajectory to: scaled_pos_joint_traj_controller/follow_joint_trajectory

Now you can use moveIt! for Inverse Kinematics with a real UR5e arm:

In [ ]:
roslaunch ur5_e_moveit_config ur5_e_moveit_planning_execution.launch

In [ ]:
roslaunch ur5_e_moveit_config moveit_rviz.launch config:=true

